In [7]:
# Packages
import requests
import bs4
import re
import pandas as pd
import numpy as np
import roman

In [8]:
request_obj = requests.get('https://www.gutenberg.org/files/3800/3800-h/3800-h.htm')
request_html = bs4.BeautifulSoup(request_obj.text, 'html.parser')

In [9]:
def_1_unclean = request_html.findAll('p')[0:11]
def_1_clean = list()
for line in range(len(def_1_unclean)):
    if len(re.findall('Explanation—', str(def_1_unclean[line]))) != 0:
        html_tag = re.compile('<.*?>') 
        back_slash = re.compile('\\r|\\n')
        str_combined = (str(def_1_unclean[line-1]) + ' ' + str(def_1_unclean[line]))
        def_1_clean.append(re.sub(back_slash, ' ' , re.sub(html_tag, '', str_combined)))
    elif 'footnote' not in str(def_1_unclean[line]) and len(re.findall('Explanation—', str(def_1_unclean[line+1]))) == 0:
            html_tag = re.compile('<.*?>') 
            back_slash = re.compile('\\r|\\n')
            def_1_clean.append(re.sub(back_slash, ' ' , re.sub(html_tag, '', str(def_1_unclean[line]))))
part_1_definitions = pd.DataFrame({'text': def_1_clean, 'type': 'Definition', 'number': list(range(1, len(def_1_clean)+1)), 'part': np.repeat(1, len(def_1_clean))})
axiom_1_unclean = request_html.findAll('p')[11:18]
axiom_1_clean = list()
for line in range(len(axiom_1_unclean)):
    html_tag = re.compile('<.*?>') 
    back_slash = re.compile('\\r|\\n')
    axiom_1_clean.append(re.sub(back_slash, ' ' , re.sub(html_tag, '', str(axiom_1_unclean[line]))))
part_1_axiom = pd.DataFrame({'text': axiom_1_clean, 'type': 'Axiom', 'number': list(range(1, len(axiom_1_clean)+1)),'part': np.repeat(1, len(axiom_1_clean))})
part_1_prop_unclean = str(request_html.findAll('p')[18:len(request_html.findAll('p'))])
html_tag = re.compile('<.*?>') 
back_slash = re.compile('\\r|\\n')
prop_1_no_tag = re.sub(back_slash, ' ' , re.sub(html_tag, '', str(part_1_prop_unclean)))
prop_1_clean_split = re.split('PROP', prop_1_no_tag)[1:37]
part_1_prop = pd.DataFrame({'text': prop_1_clean_split, 'type': 'Proposition', 'number':  list(range(1, len(prop_1_clean_split)+1)), 'part': np.repeat(1, len(prop_1_clean_split))})
part_1 = pd.concat([part_1_definitions, part_1_axiom, part_1_prop])

#### Find Definition Reference and Clean

In [10]:
prop_single_def = list()
for i in range(len(part_1_prop)):
    prop_single_def.append((re.findall(r'(?<=\Def. )(.*?)(?=\.)', part_1_prop.text[i])))

In [11]:
prop_multiple_def = list()
for props in range(len(part_1_prop)):
    if len(re.findall(r'(?<=\Deff. )(.*?)(?<=\. and )(.*?)(?=\.)', part_1_prop.text[props]))>0:
        str_unclean = list(re.findall(r'(?<=\Deff. )(.*?)(?<=\. and )(.*?)(?=\.)', part_1_prop.text[props])[0])
        str_clean = list()
        for string in range(len(str_unclean)):
            if 'and' in str_unclean[string]:
                str_clean.append(re.sub(r'. and ', '', str_unclean[string]))
            else:
                str_clean.append(str_unclean[string])
        prop_multiple_def.append(str_clean)
    else:
        prop_multiple_def.append(re.findall(r'(?<=\Deff. )(.*?)(?<=\. and )(.*?)(?=\.)', part_1_prop.text[props]))

In [12]:
total_prop_def_list = [prop_multiple_def[i] + prop_single_def[i] for i in range(len(prop_multiple_def))]

### How many times each proposition referenced a definition?

In [13]:
total_prop_def_list

[['iii', 'v'],
 ['iii'],
 [],
 ['iii', 'v'],
 ['iii', 'vi'],
 ['iii', 'v', 'iii'],
 [],
 ['ii'],
 ['iv'],
 ['iv', 'iii'],
 ['vi'],
 ['iv'],
 [],
 ['vi', 'vi'],
 ['iii', ' v'],
 ['vi'],
 ['vii'],
 [],
 ['vi', 'viii', ' iv'],
 ['viii'],
 ['ii'],
 [],
 ['v', 'viii'],
 ['i'],
 ['v'],
 [],
 [],
 ['iii'],
 [],
 [],
 ['v', 'vi'],
 [' vii'],
 ['vii'],
 [],
 [],
 []]

In [14]:
pd.DataFrame({'text': prop_1_clean_split, 'type': 'Proposition', 'number':  \
list(range(1, len(prop_1_clean_split)+1)), 'part': np.repeat(1, len(prop_1_clean_split)),\
    'def_ref': [len(total_prop_def_list[i]) for i in range(len(total_prop_def_list))]}).head(5)

,text,type,number,part,def_ref
0,. I. Substance is by nature prior to its modi...,Proposition,1,1,2
1,". II. Two substances, whose attributes are di...",Proposition,2,1,1
2,. III. Things which have nothing in common ca...,Proposition,3,1,0
3,. IV. Two or more distinct things are disting...,Proposition,4,1,2
4,. V. There cannot exist in the universe two o...,Proposition,5,1,2


### How Many Times Each Definition Was Referenced?

In [15]:
def flatten(xss):
    return [x for xs in xss for x in xs]
def_reference = list()
prop_def_list_flat = flatten(total_prop_def_list)
for i in range(len(prop_def_list_flat)):
    no_space = re.sub(r'\s+', '', prop_def_list_flat[i])
    def_reference.append(roman.fromRoman(no_space.upper()))
pd.DataFrame({'text': def_1_clean, 'type': 'Definition', 'number': list(range(1, len(def_1_clean)+1)), \
    'part': np.repeat(1, len(def_1_clean)), 'referenced' : np.unique(def_reference, return_counts = True)[1]}).head(5)

,text,type,number,part,referenced
0,"I. By that which is self—caused, I mean tha...",Definition,1,1,1
1,II. A thing is called finite after its kind...,Definition,2,1,2
2,"III. By substance, I mean that which is in ...",Definition,3,1,9
3,"IV. By attribute, I mean that which the int...",Definition,4,1,4
4,"V. By mode, I mean the modifications[1] of ...",Definition,5,1,7


### Axioms

In [16]:
axiom_ref = [re.findall(r'(?<=Ax. )(.*?)(?=\.)', prop_1_clean_split[i]) for i in range(len(prop_1_clean_split))]
axiom_ref_num = [len(axiom_ref[i]) for i in range(len(axiom_ref))]

In [17]:
pd.DataFrame({'text': prop_1_clean_split, 'type': 'Proposition', 'number':  \
list(range(1, len(prop_1_clean_split)+1)), 'part': np.repeat(1, len(prop_1_clean_split)),\
    'def_ref': [len(total_prop_def_list[i]) for i in range(len(total_prop_def_list))],\
    'axiom_ref': axiom_ref_num}).head(5)

,text,type,number,part,def_ref,axiom_ref
0,. I. Substance is by nature prior to its modi...,Proposition,1,1,2,0
1,". II. Two substances, whose attributes are di...",Proposition,2,1,1,0
2,. III. Things which have nothing in common ca...,Proposition,3,1,0,2
3,. IV. Two or more distinct things are disting...,Proposition,4,1,2,2
4,. V. There cannot exist in the universe two o...,Proposition,5,1,2,0


In [18]:
axiom_reference = list()
for i in range(0,len(flatten(axiom_ref))):
        axiom_no = flatten(axiom_ref)[i]
        axiom_reference.append(roman.fromRoman(axiom_no.upper()))
pd.DataFrame({'text': axiom_1_clean, 'type': 'Axiom', 'number': list(range(1, len(axiom_1_clean)+1)), \
    'part': np.repeat(1, len(axiom_1_clean)), 'referenced' : [axiom_reference.count(i) for i in range(0, 7)]}).head(5)

,text,type,number,part,referenced
0,"I. Everything which exists, exists either i...",Axiom,1,1,0
1,II. That which cannot be conceived through ...,Axiom,2,1,5
2,III. From a given definite cause an effect ...,Axiom,3,1,0
3,IV. The knowledge of an effect depends on a...,Axiom,4,1,0
4,V. Things which have nothing in common cann...,Axiom,5,1,4


### Propositions References

In [19]:
prop_ref = [re.findall(r'(?<=Prop. )(.*?)(?=\.|,|\))', prop_1_clean_split[i]) for i in range(len(prop_1_clean_split))]
prop_ref_num = [len(prop_ref[i]) for i in range(len(axiom_ref))]
prop_reference = list()
for i in range(0,len(flatten(prop_ref))):
        prop_ref_flat = flatten(prop_ref)
        no_space = re.sub(r'\s+', '', prop_ref_flat[i])
        prop_reference.append(roman.fromRoman(no_space.upper()))

In [20]:
prop_1_data = pd.DataFrame({'text': prop_1_clean_split, 'type': 'Proposition', 'number':  \
list(range(1, len(prop_1_clean_split)+1)), 'part': np.repeat(1, len(prop_1_clean_split)),\
    'def_ref': [len(total_prop_def_list[i]) for i in range(len(total_prop_def_list))],\
    'axiom_ref': axiom_ref_num, 'prop_ref': prop_ref_num, 'ded_index': \
        [1/(1+prop_ref_num[i] + (axiom_ref_num[i] + len(total_prop_def_list[i]))) for i in range(len(prop_ref_num))]})

In [21]:
pd.DataFrame({'text': prop_1_clean_split, 'type': 'Proposition', 'number': list(range(1, len(prop_1_clean_split)+1)), \
    'part': np.repeat(1, len(prop_1_clean_split)), 'referenced' : [prop_reference.count(i) for i in range(len(prop_1_clean_split))]})

,text,type,number,part,referenced
0,. I. Substance is by nature prior to its modi...,Proposition,1,1,0
1,". II. Two substances, whose attributes are di...",Proposition,2,1,1
2,. III. Things which have nothing in common ca...,Proposition,3,1,3
3,. IV. Two or more distinct things are disting...,Proposition,4,1,1
4,. V. There cannot exist in the universe two o...,Proposition,5,1,2
5,. VI. One substance cannot be produced by ano...,Proposition,6,1,4
6,. VII. Existence belongs to the nature of sub...,Proposition,7,1,4
7,. VIII. Every substance is necessarily infini...,Proposition,8,1,12
8,". IX. The more reality or being a thing has, ...",Proposition,9,1,3
9,. X. Each particular attribute of the one sub...,Proposition,10,1,0


In [77]:
def_ref_p1 =[['I Def.' + ' ' + re.sub('\s+', '', i) for i in total_prop_def_list[j]] for j, tuple in enumerate(total_prop_def_list)]
axiom_ref_p1 = [['I Axiom.' + ' ' + re.sub('\s+', '', i) for i in axiom_ref[j]] for j, tuple in enumerate(total_prop_def_list)]
prop_ref_p1 =[['I Prop.' + ' ' + re.sub('\s+', '', i) for i in prop_ref[j]] for j, tuple in enumerate(total_prop_def_list)]
p1_ref_list_complete = []
p1_len_ref_list_complete = []
for index, tuple in enumerate(total_prop_def_list):
    complete_ref_list= def_ref_p1[index] + axiom_ref_p1[index] + prop_ref_p1[index]
    p1_ref_list_complete.append(complete_ref_list)
    prop_name = 'I Prop.' + ' ' + str(roman.toRoman(index + 1)).lower()
    p1_len_ref_list_complete.append(list(np.repeat(prop_name, len(complete_ref_list))))


In [78]:
p1_len_ref_list_complete

[['I Prop. i', 'I Prop. i'],
 ['I Prop. ii'],
 ['I Prop. iii', 'I Prop. iii'],
 ['I Prop. iv', 'I Prop. iv', 'I Prop. iv', 'I Prop. iv'],
 ['I Prop. v', 'I Prop. v', 'I Prop. v', 'I Prop. v', 'I Prop. v'],
 ['I Prop. vi',
  'I Prop. vi',
  'I Prop. vi',
  'I Prop. vi',
  'I Prop. vi',
  'I Prop. vi',
  'I Prop. vi'],
 [],
 ['I Prop. viii',
  'I Prop. viii',
  'I Prop. viii',
  'I Prop. viii',
  'I Prop. viii',
  'I Prop. viii',
  'I Prop. viii'],
 ['I Prop. ix'],
 ['I Prop. x', 'I Prop. x'],
 ['I Prop. xi',
  'I Prop. xi',
  'I Prop. xi',
  'I Prop. xi',
  'I Prop. xi',
  'I Prop. xi'],
 ['I Prop. xii',
  'I Prop. xii',
  'I Prop. xii',
  'I Prop. xii',
  'I Prop. xii',
  'I Prop. xii',
  'I Prop. xii',
  'I Prop. xii'],
 ['I Prop. xiii', 'I Prop. xiii', 'I Prop. xiii', 'I Prop. xiii'],
 ['I Prop. xiv',
  'I Prop. xiv',
  'I Prop. xiv',
  'I Prop. xiv',
  'I Prop. xiv',
  'I Prop. xiv'],
 ['I Prop. xv',
  'I Prop. xv',
  'I Prop. xv',
  'I Prop. xv',
  'I Prop. xv',
  'I Prop. xv',
  '

In [83]:
node_list = pd.DataFrame({'target': flatten(p1_len_ref_list_complete), 'source': flatten(p1_ref_list_complete), 'weight': np.repeat(1, \
    len(flatten(p1_ref_list_complete)))})

node_list

,target,source,weight
0,I Prop. i,I Def. iii,1
1,I Prop. i,I Def. v,1
2,I Prop. ii,I Def. iii,1
3,I Prop. iii,I Axiom. v,1
4,I Prop. iii,I Axiom. iv,1
...,...,...,...
141,I Prop. xxxvi,I Prop. xxxii,1
142,I Prop. xxxvi,I Prop. xvi,1
143,I Prop. xxxvi,I Prop. xvi,1
144,I Prop. xxxvi,I Prop. xxv,1


In [101]:
#graph 
import matplotlib.pyplot as plot 
import networkx
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
output_notebook()


Loading BokehJS ...

In [102]:
node_edge = networkx.from_pandas_edgelist(node_list, 'source', 'target', 'weight')
degrees = dict(networkx.degree(node_edge))
networkx.set_node_attributes(node_edge, name='degree', values=degrees)
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(node_edge)])
networkx.set_node_attributes(node_edge, name='adjusted_node_size', values=adjusted_node_size)

In [128]:
list(networkx.degree(node_edge))

[('I Def. iii', 8),
 ('I Prop. i', 3),
 ('I Def. v', 7),
 ('I Prop. ii', 4),
 ('I Axiom. v', 1),
 ('I Prop. iii', 3),
 ('I Axiom. iv', 4),
 ('I Prop. iv', 5),
 ('I Axiom. i', 5),
 ('I Prop. v', 8),
 ('I Def. vi', 6),
 ('I Prop. vi', 9),
 ('I Def. ii', 2),
 ('I Prop. viii', 6),
 ('I Prop. vii', 5),
 ('I Def. iv', 4),
 ('I Prop. ix', 1),
 ('I Prop. x', 4),
 ('I Prop. xi', 12),
 ('I Prop. xii', 8),
 ('I Prop. xiii', 6),
 ('I Prop. xiv', 12),
 ('I Prop. xv', 16),
 ('I Prop. xvi', 8),
 ('I Def. vii', 3),
 ('I Prop. xvii', 7),
 ('I Prop. xviii', 3),
 ('I Prop. xix', 7),
 ('I Def. viii', 3),
 ('I Prop. xx', 2),
 ('I Prop. xxi', 5),
 ('I Prop. xxiii', 6),
 ('I Prop. xxii', 1),
 ('I Def. i', 1),
 ('I Prop. xxiv', 4),
 ('I Prop. xxv', 5),
 ('I Prop. xxviii', 7),
 ('I Prop. xxvi', 2),
 ('I Prop. xxix', 10),
 ('I Prop. xxvii', 1),
 ('I Axiom. vi', 1),
 ('I Prop. xxx', 3),
 ('I Prop. xxxi', 3),
 ('I Prop. xxxii', 5),
 ('I Prop. xxxiii', 6),
 ('I Prop. xxxiv', 3),
 ('I Prop. xxxvi', 5)]

In [106]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Part 1 Ethics Spinoza Connections'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Proposition", "@index"),
        ("Number of Reference", "@degree")
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html\
network_graph = from_networkx(node_edge, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

show(plot)
